# Тема: "Парсинг данных. HTML, Beautiful Soap"

### Задание 1

Необходимо собрать информацию о вакансиях на вводимую должность (используем **input** или через аргументы получаем должность) с сайтов **HeadHanter** (обязательно) и/или **Superjob** (по желанию).

Приложение должно анализировать несколько страниц сайта (также вводим через **input** или аргументы). Получившийся список должен содержать в себе минимум:

* Наименование вакансии.

* Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).

* Ссылку на саму вакансию.

* Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью **DataFrame** через **Pandas**. Сохраните результат в файл **json** либо **csv**.

#### Ответ:

In [1]:
# Импорт нужных библиотек:
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
# Обозначение URL сайтов:

# HeadHanter:
URL_HH = 'https://hh.ru/search/vacancy'

# SuperJob:
HOST_SJ = 'https://www.superjob.ru'
URL_SJ = 'https://www.superjob.ru/vacancy/search'

In [3]:
# Устновка User-Agent:
HEADER = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
          AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

In [4]:
# Функция запроса HTML-страницы по URL сайта:
def get_html(url, params=''):
    response = requests.get(url, headers = HEADER, params = params)
    
    return response

In [5]:
# Функция запроса контента с HTML-страницы HeadHanter:
def get_hh_content(response):
    soup = bs(response, 'lxml')
    items = soup.find_all('div', class_ = 'serp-item')
    
    hh_vacancy = []
    for item in items:
        hh_vacancy.append(
            {'site': 'HeadHanter', # Название сайта
             'title': item.find('a', {'class': 'serp-item__title'}).get_text(), # Название вакансии
             'link': item.find('a', {'class': 'serp-item__title'})['href'], # Ссылка на вакансию
             'salary': item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}), # Зарплата
             'city': item.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).contents[0].strip(', '), # Город
             'organization': item.find('a', {'class': 'bloko-link_kind-tertiary'}), # Название компании
             'note': item.find('div', {'class': 'vacancy-serp-item__label'})}) # Примечание

    # Упорядочение полученных данных:
    for i in hh_vacancy:
        # Salary
        try:
            i['salary'] = i['salary'].text
        except:
            i['salary'] = None
        if i['salary']:
            salary_list = i['salary'].split(' ')
            if salary_list[0] == 'от':
                i['salary_min'] = salary_list[1]
                i['salary_max'] = None
            elif salary_list[0] == 'до':
                i['salary_min'] = None
                i['salary_max'] = salary_list[1]
            else:
                i['salary_min'] = salary_list[0]
                i['salary_max'] = salary_list[2]
            i['salary_currency'] = salary_list[-1]
        else:
            i['salary_min'] = None
            i['salary_max'] = None
            i['salary_currency'] = None
        i.pop('salary')
        # City
        if i['city']:
            city_list = i['city'].split(',')
            i['city'] = city_list[0]
        # Organization
        if i['organization'] != None:
            i['organization'] = i['organization'].get_text()
        # Note
        if i['note'] != None:
            i['note'] = i['note'].get_text()
    
    return hh_vacancy

In [6]:
# Главная функция парсинга сайта HeadHanter:
def parser_hh():
    post = str(input('Введите название вакансий для парсинга: '))
    pages = int(input('Введите количество страниц для парсинга: '))
    html = get_html(URL_HH)
    
    if html.status_code == 200:
        vacancy = []
        for page in range(1, pages + 1):
            print(f'Парсится hh, страница: {page}')
            html = get_html(URL_HH, params = {'text': post, 'page': page})
            vacancy.extend(get_hh_content(html.text))
        hh_result = pd.DataFrame(vacancy)
    
    else:
        hh_result = html.status_code
    
    return hh_result

parser_hh()

Введите название вакансий для парсинга: Разработчик Python
Введите количество страниц для парсинга: 2
Парсится hh, страница: 1
Парсится hh, страница: 2


,site,title,link,city,organization,note,salary_min,salary_max,salary_currency
0,HeadHanter,Программист Python,https://vladivostok.hh.ru/vacancy/76331573?fro...,Санкт-Петербург,Парконика,None,90 000,None,руб.
1,HeadHanter,Python разработчик (Django),https://vladivostok.hh.ru/vacancy/76661153?fro...,Санкт-Петербург,Stik,None,40 000,60 000,руб.
2,HeadHanter,Программист Python,https://vladivostok.hh.ru/vacancy/76785097?fro...,Москва,Enjoypro,Можно из дома,None,240 000,руб.
3,HeadHanter,Senior Python developer (Django),https://vladivostok.hh.ru/vacancy/76821518?fro...,Турция,evrone.ru,Можно из дома,None,450 000,руб.
4,HeadHanter,Middle Python Developer,https://vladivostok.hh.ru/vacancy/76820856?fro...,Москва,Филиал компании Лист Ренталс Лимитед,None,2 000,4 000,USD
5,HeadHanter,Разработчик Python,https://vladivostok.hh.ru/vacancy/76433537?fro...,Москва,Платежный сервис А3,None,150 000,None,руб.
6,HeadHanter,Python разработчик на внутренний проект (удалё...,https://vladivostok.hh.ru/vacancy/77013948?fro...,Москва,ГК Медси. Управляющая компания,Можно из дома,180 000,250 000,руб.
7,HeadHanter,Python Engineer,https://vladivostok.hh.ru/vacancy/76983666?fro...,Баку,ТРТ,Можно из дома,3 000,6 500,USD
8,HeadHanter,Разработчик Python (искусственный интеллект),https://vladivostok.hh.ru/vacancy/77038946?fro...,Москва,Корпорация ЭЛАР,Можно из дома,None,None,None
9,HeadHanter,Программист Python (middle),https://vladivostok.hh.ru/vacancy/75719539?fro...,Орел,Pentestit,Можно из дома,70 000,105 000,руб.


In [7]:
# Функция запроса контента HTML-страницы SuperJob:
def superjob_get_content(response):
    soup = bs(response, 'lxml')
    items = soup.find_all('div', {'class': 'f-test-vacancy-item'})

    sj_vacancy = []
    for item in items:
        sj_vacancy.append(
            {'site': 'SuperJob',  # Название сайта
             'title': item.find('a').get_text(),  # Название вакансии
             'link': item.find('a').get('href'),  # Ссылка на вакансию
             'salary': item.find('div', {'class': 'f-test-text-company-item-salary'}),  # Зарплата
             'city': item.find('span', {'class': 'f-test-text-company-item-location'}),  # Город
             'organization': item.find('span', {'class': 'f-test-text-vacancy-item-company-name'}), # Название компании
             'note': item.find('span', {'class': 'f-test-badge'})})  # Примечание

   # Упорядочение полученных данных:
    for v in sj_vacancy:
        # Link
        v['link'] = HOST_SJ + v['link']
        # Salary
        if v['salary'] != None:
            v['salary'] = v['salary'].get_text()
        try:
            v['salary'] = v['salary'].text
        except:
            if v['salary'] != 'По договорённости':
                salary_list = v['salary'].split('\xa0')
            if salary_list[0] == 'от':
                v['salary_min'] = salary_list[1] + salary_list[2]
                v['salary_max'] = None
            elif salary_list[0] == 'до':
                v['salary_min'] = None
                v['salary_max'] = salary_list[1] + salary_list[2]
            elif len(salary_list) == 3:
                v['salary_min'] = salary_list[0] + salary_list[1]
                v['salary_max'] = salary_list[0] + salary_list[1]
            else:
                v['salary_min'] = salary_list[0] + salary_list[1]
                v['salary_max'] = salary_list[2] + salary_list[3]
            v['salary_currency'] = salary_list[-1].split('/')[0]
        else:
            v['salary_min'] = None
            v['salary_max'] = None
            v['salary_currency'] = None
        v.pop('salary')
        # City
        if v['city'] != None:
            v['city'] = v['city'].get_text()
        # Organization
        if v['organization'] != None:
            v['organization'] = v['organization'].get_text()
        # Note
        if v['note'] != None:
            v['note'] = v['note'].get_text()
        
    return sj_vacancy

In [8]:
# Главная функция парсинга сайта SuperJob:
def parser_sj():
    POST = str(input('Введите название вакансии для парсинга: '))
    PAGES = int(input('Количество страниц для парсинга: '))
    html = get_html(URL_SJ)
    if html.status_code == 200:
        vacancy = []
        for page in range(1, PAGES + 1):
            print(f'Парсится sj, страница: {page}')
            html = get_html(URL_SJ, params = {'keywords': POST, 'page': page})
            vacancy.extend(superjob_get_content(html.text))
        sj_result = pd.DataFrame(vacancy)
    else:
        sj_result = html.status_code
    
    return sj_result

parser_sj()

Введите название вакансии для парсинга: Разработчик Python
Количество страниц для парсинга: 2
Парсится sj, страница: 1
Парсится sj, страница: 2


,site,title,link,city,organization,note,salary_min,salary_max,salary_currency
0,SuperJob,IOS/QT developer,https://www.superjob.ru/vakansii/ios-45722326....,Новосибирск,КонсалтПро,Удаленная работа,400000,— 420000,₽
1,SuperJob,Инженер-программист (специалист по работе с МИ...,https://www.superjob.ru/vakansii/inzhener-prog...,Санкт-Петербург,СПб ГБУЗ Городская поликлиника № 38,Удаленная работа,50000,— 60000,₽
2,SuperJob,Разработчик Web и приложений,https://www.superjob.ru/vakansii/razrabotchik-...,Владивосток,ФГБОУ ВО ТГМУ Минздрава России,None,80000,— 100000,₽
3,SuperJob,Fullstack веб-разработчик middle (удаленно),https://www.superjob.ru/vakansii/fullstack-veb...,Томск,None,Отклик без резюме,40000,None,₽
4,SuperJob,Специалист службы поддержки с техническими зна...,https://www.superjob.ru/vakansii/specialist-sl...,Липецк,Яндекс,Опыт не нужен,15000,None,₽
5,SuperJob,Специалист службы поддержки с техническими зна...,https://www.superjob.ru/vakansii/specialist-sl...,Новокузнецк,Яндекс,Опыт не нужен,15000,None,₽
6,SuperJob,Специалист службы поддержки с техническими зна...,https://www.superjob.ru/vakansii/specialist-sl...,Рязань,Яндекс,Опыт не нужен,15000,None,₽
7,SuperJob,Специалист службы поддержки с техническими зна...,https://www.superjob.ru/vakansii/specialist-sl...,Набережные Челны,Яндекс,Опыт не нужен,15000,None,₽
8,SuperJob,Специалист службы поддержки с техническими зна...,https://www.superjob.ru/vakansii/specialist-sl...,Астрахань,Яндекс,Опыт не нужен,15000,None,₽
9,SuperJob,Специалист службы поддержки с техническими зна...,https://www.superjob.ru/vakansii/specialist-sl...,Пенза,Яндекс,Опыт не нужен,15000,None,₽


In [9]:
# Функция объединения полученных результатов c двух сайтов:
def main_parsing():
    post = str(input('Введите название вакансии для парсинга: '))
    pages = int(input('Введите количество страниц для парсинга: '))
    HH_HTML = get_html(URL_HH)
    SJ_HTML = get_html(URL_SJ)

    if HH_HTML.status_code == 200 and SJ_HTML.status_code == 200:
        vacancy = []
        for page in range(1, pages + 1):
            print(f'Парсится общая информация, страница: {page}')
            hh = get_html(URL_HH, params = {'text': post, 'page': page})
            sj = get_html(URL_SJ, params = {'keywords': post, 'page': page})
            vacancy.extend(get_hh_content(hh.text) + superjob_get_content(sj.text))
        result = pd.DataFrame(vacancy)
    else:
        print('error')
    
    return result

In [10]:
# Вывод общей таблицы:
vacancyDF = main_parsing()
vacancyDF

Введите название вакансии для парсинга: Разработчик Python
Введите количество страниц для парсинга: 1
Парсится общая информация, страница: 1


,site,title,link,city,organization,note,salary_min,salary_max,salary_currency
0,HeadHanter,Python-разработчик в R&D отдел,https://vladivostok.hh.ru/vacancy/72687478?fro...,Москва,ООО Селфсек,None,100 000,150 000,руб.
1,HeadHanter,Программист Python,https://vladivostok.hh.ru/vacancy/76331573?fro...,Санкт-Петербург,Парконика,None,90 000,None,руб.
2,HeadHanter,Python разработчик (Django),https://vladivostok.hh.ru/vacancy/76661153?fro...,Санкт-Петербург,Stik,None,40 000,60 000,руб.
3,HeadHanter,Программист Python,https://vladivostok.hh.ru/vacancy/76785097?fro...,Москва,Enjoypro,Можно из дома,None,240 000,руб.
4,HeadHanter,Senior Python developer (Django),https://vladivostok.hh.ru/vacancy/76821518?fro...,Турция,evrone.ru,Можно из дома,None,450 000,руб.
5,HeadHanter,Middle Python Developer,https://vladivostok.hh.ru/vacancy/76820856?fro...,Москва,Филиал компании Лист Ренталс Лимитед,None,2 000,4 000,USD
6,HeadHanter,Разработчик Python,https://vladivostok.hh.ru/vacancy/76433537?fro...,Москва,Платежный сервис А3,None,150 000,None,руб.
7,HeadHanter,Python разработчик на внутренний проект (удалё...,https://vladivostok.hh.ru/vacancy/77013948?fro...,Москва,ГК Медси. Управляющая компания,Можно из дома,180 000,250 000,руб.
8,HeadHanter,Python Engineer,https://vladivostok.hh.ru/vacancy/76983666?fro...,Баку,ТРТ,Можно из дома,3 000,6 500,USD
9,HeadHanter,Разработчик Python (искусственный интеллект),https://vladivostok.hh.ru/vacancy/77038946?fro...,Москва,Корпорация ЭЛАР,Можно из дома,None,None,None


In [11]:
# Просмотр информации общей таблицы:
vacancyDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   site             60 non-null     object
 1   title            60 non-null     object
 2   link             60 non-null     object
 3   city             60 non-null     object
 4   organization     59 non-null     object
 5   note             49 non-null     object
 6   salary_min       55 non-null     object
 7   salary_max       15 non-null     object
 8   salary_currency  57 non-null     object
dtypes: object(9)
memory usage: 4.3+ KB


In [12]:
# Сохранение общих данных в файл .csv:
vacancyDF.to_csv('vacancy.csv', index = False, encoding = 'utf8')

In [13]:
# Проверка обратным преобразованием:
#df = pd.read_csv('vacancy.csv')
#df

---

END